# 运营相关数据

In [74]:
# 工作日历

In [73]:
from imports import *

In [41]:
start_time, end_time, GE_DIR,workday_list

('2018/01/01 00:00:00',
 '2021/12/30 00:00:00',
 '/Users/zhitaogao/Library/CloudStorage/OneDrive-Personal/bbd-projects/mfg_operation_model/generated',
 [1, 2, 3, 4, 5])

# 生成相关运营的基础数据

In [42]:
# 生成工作日历，按照每天看是不是在工作

In [43]:
start_time = pd.to_datetime(start_time)
end_time = pd.to_datetime(end_time)
start_date_str = datetime.date.strftime(start_time, '%Y/%m/%d')
end_date_str = datetime.date.strftime(end_time, '%Y/%m/%d')

In [44]:
start_time + datetime.timedelta(days=1)

Timestamp('2018-01-02 00:00:00')

In [61]:
calendar = pd.DataFrame()

cur_date_str = datetime.date.strftime(start_time, '%Y/%m/%d')


In [62]:
cur_date_str

'2018/01/01'

In [63]:
idx = 1

while cur_date_str != end_date_str:
    
    calendar.loc[idx, 'date'] = cur_date_str
    
    weekday = datetime.datetime.weekday(cur_date) + 1
    
    if weekday in workday_list:
        
        workday = '工作日'
    
    else:
        workday = '休息日'

    calendar.loc[idx, 'workday'] = workday
    calendar.loc[idx, 'weekday'] = weekday


    cur_date = start_time + datetime.timedelta(days=idx)
    cur_date_str = datetime.date.strftime(cur_date, '%Y/%m/%d')
    
    idx += 1

In [64]:
calendar

,date,workday,weekday
1,2018/01/01,工作日,4.0
2,2018/01/02,工作日,2.0
3,2018/01/03,工作日,3.0
4,2018/01/04,工作日,4.0
5,2018/01/05,工作日,5.0
...,...,...,...
1455,2021/12/25,休息日,6.0
1456,2021/12/26,休息日,7.0
1457,2021/12/27,工作日,1.0
1458,2021/12/28,工作日,2.0


In [70]:
## 保存生成的日历
calendar.to_csv(os.path.join(GE_DIR, 'calendar.csv'), index=False)

# 生成当日工作时间

In [71]:
shift = pd.read_csv(os.path.join(BASE_DIR,'shift_model.csv'))

In [72]:
shift

,shift,start_time,end_time,current
0,A,7:00,18:30,1
1,B,19:00,6:30,1


In [68]:
# hourly plan

step = 60 # minutes

hourly_plan = pd.DataFrame()

start_time = pd.to_datetime(start_time)
end_time = pd.to_datetime(end_time)

start_time_hour = start_time
idx = 0

while start_time_hour < end_time:
    print(start_time)

    hourly_plan.loc[idx, 'start_time'] = start_time_hour
    hourly_plan.loc[idx, 'end_time'] = start_time_hour + pd.Timedelta(minutes=step)

    # which shift
    shift_idx = shift[(shift['start_time'] <= start_time) & (shift['end_time'] > start_time)].index[0]
    

    start_time = start_time + pd.Timedelta(minutes=step)

    idx += 1


2018-01-01 00:00:00


TypeError: '<=' not supported between instances of 'str' and 'Timestamp'

In [ ]:
start_time, end_time

(Timestamp('2021-12-30 00:00:00'), Timestamp('2021-12-30 00:00:00'))

In [ ]:
hourly_plan

""


# 停线相关数据  equipment_status

In [ ]:
# 设备状态
equipment = pd.read_csv(os.path.join(base_dir_abs, 'equipment.csv'),sep='\t', encoding='utf-8')

In [ ]:
equipment.to_csv(os.path.join(base_dir_abs, 'equipment.csv'),index=False)

,line,station,equipment_number,equipment_name
0,1,OP1,1,OP1-1
1,1,OP1,2,OP1-2
2,1,OP1,3,OP1-3
3,1,OP1,4,OP1-4
4,1,OP1,5,OP1-5
...,...,...,...,...
107,2,OP17,1,OP17-1
108,2,OP18,1,OP18-1
109,2,OP18,2,OP18-2
110,2,OP18,3,OP18-3


In [ ]:
equipment = pd.read_csv(os.path.join(base_dir_abs, 'equipment.csv'),
                sep='\t', 
                encoding='utf-8')

In [ ]:
# 停线时间 按照每小时为颗粒度生成，平均会停线的大概是ratio
equipment_downtime_ratio = 0.1
equipment_downtime_rand = 2